# Tests MSSQL Python

Frist create a .env file:

```env
MSSQL_HOST="..."
MSSQL_PORT=1433
MSSQL_DATABASE="..."
MSSQL_SCHEMA="dbo"
MSSQL_USERNAME="..."
MSSQL_PASSWORD="..."
```

In [1]:
%reload_ext dotenv
%dotenv

import os

config = {
    "host": os.getenv("MSSQL_HOST"),
    "port": int(os.getenv("MSSQL_PORT") or 0),
    "database": os.getenv("MSSQL_DATABASE"),
    "schema": os.getenv("MSSQL_SCHEMA") or "dbo",
    "username": os.getenv("MSSQL_USERNAME"),
    "password": os.getenv("MSSQL_PASSWORD"),
}

## PyODBC

**Microsoft ODBC Driver for SQL Server is required before to install pyodbc**.

Please follow the instructions to install the driver for your OS. See the [docs](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server) for details.

To install run this command:
```bash
poetry add pyodbc
```

In [ ]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["host"]};PORT={config["port"]};'
    f'DATABASE={config["database"]};'
    f'UID={config["username"]};'
    f'PWD={config["password"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        rows = cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA='dbo' AND TABLE_TYPE='BASE TABLE';")
        display(rows.fetchall())

## PyTDS

To install run this command:

```bash
poetry add python-tds
```

If you want to use TLS you should also install pyOpenSSL package:

```bash
poetry add pyOpenSSL
```

For a better performance install bitarray package too:

```bash
poetry add bitarray
```

To use Kerberos on non-Windows platforms (experimental) install kerberos package:

```bash
poetry add kerberos
```

## MiniAB

**miniab** requires duckdb and pyarrow and they should be installed before to use it.

To install duckdb and pyarrow run this command:

```bash
poetry add duckdb pyarrow
```

In [ ]:
import duckdb
import miniab as mab

display(mab.get_available_connectors())

source: mab.Source = mab.get_source(
    "source-mssql",
    config = config,
    sync = True
)

source.select_all_streams()
result = source.read(force_full_refresh=True)

tteam_tbl = result.to_arrow("TTeam")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 10;").show()

['source-snowflake', 'source-mssql']

Sync Progress: MsSqlSource -> DuckdbCache
Started reading from source at 15:24:29
 * Received records for 32 streams:
  - 71 Corner_Reference                  
  - 117 Dic_Area2Brick                  
  - 123 Dic_Area2Region                  
  - 23 Dic_BusinessLogic                  
  - 61,998 Dic_Customer                  
  - 19 Dic_CustomerType                  
  - 59 Dic_FIE_FIN2STMS_ProductMapping                  
  - 56 Dic_FIE_FIN2STMS_SpiritCodeMapping                  
  - 8 Dic_FIE_KDFile2STMS_ProductMapping                  
  - 10 Dic_Parameter                  
  - 832 Dic_Price                  
  - 150 Dic_Product                  
  - 506 Dic_ProductPrice                  
  - 218 Dic_ProductType                  
  - 2 Global_Configuration                  
  - 5 Hierarchy                  
  - 158 IS_Satisfaction_Survey_FY2223                  
  - 41,625 Log_ScheduledProcess                  
  - 25,566 Raw_DKSH                  
  - 22 Ref_Activity2Score        

## PyAirByte

PyAirByte requires docker and it should be installed before to use it.

To install PyAirByte run this command:

```bash
poetry add airbyte
```

In [ ]:
import duckdb
import airbyte as ab

display(ab.get_available_connectors())

source: ab.Source = ab.get_source(
    "source-mssql",
    config = config
)

source.select_all_streams()
result = source.read()

tteam_tbl = result.to_arrow("")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 10;").show()